#### Installs ----- commented in or out and restart kernel after completed

In [1]:
# !pip install openai
# !pip install --upgrade openai
# !pip install --upgrade pip
# !pip install GitPython
# !pip install beautifulsoup4

# image processing
#!pip install pillow
# !pip install pydantic
# !pip install python-dotenv

#### <u><b>Libraries</u></b>

In [1]:
from getpass import getpass
import openai
from openai import OpenAI
import os
from git import Repo
#used for setting up paths and parent dir to github repository
from pathlib import Path
import shutil
from bs4 import BeautifulSoup as Soup
from PIL import Image
from dotenv import load_dotenv
from datetime import datetime as dt # for formatting date returned with images
import requests                # for downloading images from URLs

#### Set Folders

In [2]:
#git repo folder - the folder that holds the changes and commits
PATH_TO_BLOG_REPO = Path('C:\\Users\\DARiN\\Documents\\Python-JL\\Github\\Blog-Generator.github.io\\.git')
#main folder to inject the index.html files
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent
#content subfolder where we will put the blog content
PATH_TO_CONTENT = PATH_TO_BLOG/"content"
#creates content folder
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

#### <u><b>Set API</u></b>

In [12]:
inputuser=input('enter your OpenAI username: ')
os.environ["OPENAI_API_KEY"] = ''
load_dotenv('database.env') #loads file you must make called "database.env" which contains "openaikey=" followed by your open ai api key
openai.api_key=os.getenv('openaikey')
client = OpenAI(api_key=openai.api_key)

enter your OpenAI username:  caliskate04


#### Define Git Push

In [14]:
def update_blog(commit_message='Updates blog - notebook final edits'):
        #tells GitPython the Repo Location
        repo = Repo(PATH_TO_BLOG_REPO)
        #git add . (everything)
        repo.git.add(all=True)
        #git commit with some message (-m) "updates blog"
        repo.index.commit(commit_message)
        #git push
        origin = repo.remote(name='origin')
        origin.push()

#### Create Blog Function Defined Here

In [15]:
#blog_utils
def create_new_blog(title,content,cover_image):
    cover_image = Path(cover_image)

    files = len(list(PATH_TO_CONTENT.glob("*.html")))     #grab all html files and count them
    new_title = f"{files+1}.html" #creates file and folder name by counting up by 1
    path_to_new_content = PATH_TO_CONTENT/new_title #getting the file and making sure in correct dir
    
    shutil.copy(cover_image,PATH_TO_CONTENT) #checks to make sure file name does not exist - prevents error or overwriting existing files
    print(new_title)
    if not os.path.exists(path_to_new_content):
        #write a new html file
        with open(path_to_new_content,"w") as f:
            #write commands for each line of the blog
            #write head of html open and close
            f.write("<!DOCTYPE HTML>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title>{title}</title>\n")
            f.write("</head>\n")
            #write body open and close with title, image and content
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            #open ai provides \n but we also want to replace that to include break calls
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print('Blog Created')
            return path_to_new_content
    else:
            raise FileExistsError("File name already exists")

In [16]:
#get an index that lists all of the blog posts
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

str(soup)

'<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n<title>\n   Document\n  </title>\n</head>\n<body>\n<h1>\n   My BLOG Home page\n  </h1>\n<a href="index.html">\n   Home\n  </a>\n<a href="content\\4.html">\n   4\n  </a>\n<a href="content\\5.html">\n   5\n  </a>\n<a href="content\\6.html">\n   6\n  </a>\n<a href="content\\7.html">\n   7\n  </a>\n<a href="content\\8.html">\n   8\n  </a>\n<a href="content\\9.html">\n   9\n  </a>\n<a href="content\\10.html">\n   10\n  </a>\n<a href="content\\11.html">\n   11\n  </a>\n<a href="content\\12.html">\n   12\n  </a>\n<a href="content\\13.html">\n   13\n  </a>\n<a href="content\\14.html">\n   14\n  </a>\n<a href="content\\15.html">\n   15\n  </a>\n<a href="content\\16.html">\n   16\n  </a>\n<a href="content\\17.html">\n   17\n  </a>\n<a href="content\\18.html">\n   18\n  </a>\n<a href="content\\19.html">\n   19\n  </a>\n<a href="content\\22.html">\n   22\n  </a>\n<a href="co

In [17]:
#check for duplicate links, using href (the anchor tag for index.html)
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links] #1.html,2.html,3.html
    content_path=str(Path(*path_to_new_content.parts[-2:])) # checks for index of 2 until the end ... would look like C:/UIsers/marci/file.../1.html
    return content_path in urls

In [18]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/'index.html') as index:
        soup = Soup(index.read()) #looks for all anchor tags then finds the last link
        
    links = soup.find_all('a')
    last_link = links[-1]

    if check_for_duplicate_links(path_to_new_content,links):
        raise ValueError("Link already exists!")
        # a lot of this takes advantage of what beautiful soup already has, need to familiarize what is has if you're working with html and the web
        #finds the path to the new content
    link_to_new_blog = soup.new_tag("a",href=Path(*path_to_new_content.parts[-2:]))
        #convert the path into a string and instert into a new blog
    link_to_new_blog.string = path_to_new_content.name.split('.')[0]
    last_link.insert_after(link_to_new_blog)

    with open(PATH_TO_BLOG/'index.html','w') as f:
        f.write(str(soup.prettify(formatter='html'))) #prettify with separate long string into new lines

#### Defining Text Prompt

In [19]:
def create_prompt(title):
    prompt="""
    Biography:
    My name is Darin and I am a Python student.

    Blog
    Title: {}
    tags: tech, python, coding, AI, machine learning
    Summary: I learn about what Python can hold for the future of AI. Please do not exceed 2 paragraphs in total response length.
    Full text: """.format(title) #070224 added 'Do not exceed 2 paragraphs.' based on max token length of 250
    return prompt

title = "Python and the Future of AI" #fixed format 070224
print(create_prompt(title))


    Biography:
    My name is Darin and I am a Python student.

    Blog
    Title: Python and the Future of AI
    tags: tech, python, coding, AI, machine learning
    Summary: I learn about what Python can hold for the future of AI. Please do not exceed 2 paragraphs in total response length.
    Full text: 


#### Chat GPT 3.5 API call - (rerun from here to create new blog)

In [22]:
# old format - as needed, for reference
# completion=client.completions.create(
#                 model='gpt-3.5-turbo-instruct',
#                 prompt=create_prompt(title),
#                 max_tokens=250, #length of output
#                 temperature=0.7, # 0 to 1, only change top p or temperature , 0 is the most probable, 1 takes more risks
#                 top_p = 1.0, #only change top p or temperature, 1 is default, 0.1 is top 10% most probable
#                 frequency_penalty=0, #-2 to 2, positive value penalizes new tokens if they exist, decreaseing probabiolity of repeating text
#                 presence_penalty=0, #-2 to 2, positive penalize new tokens if they appear, but thinks about tokens, value too high creates low probability
#                 n = 1, #number or results to be rewturned, 1 is default, use when temp is zero to min repeating results
#                 # stop = ['/n'],
# )         
# blog_content=completion.choices[0].text
# print(blog_content)

In [23]:
#new api call format
def system_prompt():
    return "You are a helpful assistant to write blogposts."
def get_blog_from_openai(blog_title):
    response=client.chat.completions.create(
                            model="gpt-3.5-turbo",
                            messages=[
                                {"role": "system",
                                  "content": system_prompt()},
                                {"role": "user", "content": create_prompt(blog_title)},
                            ],
                                temperature=0.7,
                                max_tokens=250,
)         
    return response.choices[0].message.content
blog_content = get_blog_from_openai(title)
print(blog_content)

Python has emerged as a powerhouse in the field of artificial intelligence and machine learning due to its simplicity, versatility, and vast array of libraries. As a Python student, I am excited about the endless possibilities this programming language offers in shaping the future of AI. Python's ease of use and readability make it an ideal choice for both beginners and experienced developers looking to delve into AI and machine learning projects.

With Python's rich ecosystem of libraries such as TensorFlow, scikit-learn, and PyTorch, developers can easily implement complex algorithms and models for AI applications. Its strong community support and active development further solidify Python's position as a leading language in the AI landscape. As I continue to explore Python's capabilities in AI, I am confident that this dynamic language will play a pivotal role in driving innovation and advancements in the field of artificial intelligence.


#### Defining Image Prompt

In [ ]:
def dalle3_prompt(title):
    prompt=f"Sci-fi art showing a movie scene with {title}, extremely detailed and mesmerizing."
    return prompt #added extremely detailed and mesmerizing 070224

#### Dalle3 API Call

In [ ]:
image_params = {
 "model": "dall-e-3",  # Defaults to dall-e-2
 "n": 1,               # Between 2 and 10 is only for DALL-E 2
 "size": "1024x1024",  # 256x256, 512x512 only for DALL-E 2 - not much cheaper - 1024x1024, 1024x1792 or 1792x1024 available for DALL-E 3
 "prompt": dalle3_prompt(title),     # DALL-E 3: max 4000 characters, DALL-E 2: max 1000
 "user": inputuser,     # pass a customer ID to OpenAI for abuse monitoring
}
images_response = client.images.generate(**image_params)

In [ ]:
#naming image and getting url
dname = dt.now().strftime('%m-%d-%Y %H:%M:%S').replace("-",'').replace(":",'').replace(' ','-')
img_filename = 'BLOG-image_' + dname +'.png'
image_url=images_response.data[0].url

#downloading image from url
with open(img_filename,'wb') as f:
    shutil.copyfileobj(requests.get(image_url, stream=True).raw, f)
print(img_filename,'downloaded')

In [ ]:
#check image - rerun DALLE API call if not satisfied - notebook use only
Image.open(img_filename).resize((300,300))

In [ ]:
#defining new path
#combine create blog! - added new auto filename 070224
path_to_new_content=create_new_blog(title,blog_content, img_filename) #main, func from blog utils
write_to_index(path_to_new_content) # main, func from blog utils

In [ ]:
#push to github
update_blog()